In [2]:

import pandas as pd
import pickle
import networkx as nx
import sys
import os

sys.path.append(os.path.abspath('..'))
from src.features import get_pagerank, get_approx_betweenness, get_clustering_coefficient

# 1. Load the graph from the pickle file
# Make sure the path is correct relative to where you run this script
with open("../data/processed/amazon_graph.pickle", "rb") as f:
    G_loaded = pickle.load(f)



Configurazione completata. Output sarà:  ../data/processed/amazon_graph.pickle
Inizio lettura file: ../data/amazon-meta.txt
Nodi validi trovati (filtrati per gruppo): 542664
Costruzione del grafo NetworkX (DIRETTO)...
Grafo costruito.
Nodi Totali: 542664
Archi Totali: 1231370
Estrazione della Componente Debolmente Connessa Gigante (LWCC)...
GCC estratta.
Nodi GCC: 334843
Archi GCC: 1145014
Salvataggio in ../data/processed/amazon_graph.pickle...
Fatto! File .pickle generato con successo.


In [3]:
# 2. Pass the LOADED graph object to the function
pagerank_scores = get_pagerank(G_loaded,0.9)


df_pagerank = pd.DataFrame(list(pagerank_scores.items()), columns=['ASIN', 'PageRank'])
display(df_pagerank.head(5))
# Save to CSV
df_pagerank.to_csv("../data/processed/pagerank_scores.csv", index=False)


# Now you can use the scores
print(f"Computed PageRank for {len(pagerank_scores)} nodes.")

Calcolo PageRank su 334843 nodi...


,ASIN,PageRank
0,0827229534,8.386325e-06
1,0738700797,8.178738e-06
2,0842328327,6.473884e-07
3,1577943082,1.558228e-06
4,0486220125,3.262809e-07


Computed PageRank for 334843 nodes.


In [4]:
bet_scores = get_approx_betweenness(G_loaded)


df_bet = pd.DataFrame(list(bet_scores.items()), columns=['ASIN', 'PageRank'])
display(df_bet.head(5))
# Save to CSV
df_bet.to_csv("../data/processed/bet_scores.csv", index=False)


# Now you can use the scores
print(f"Computed bet cent for {len(bet_scores)} nodes.")

KeyboardInterrupt: 

In [ ]:
clus_scores = get_clustering_coefficient(G_loaded)


df_clus = pd.DataFrame(list(clus_scores.items()), columns=['ASIN', 'PageRank'])
display(df_clus.head(5))
# Save to CSV
df_clus.to_csv("../data/processed/clus_scores.csv", index=False)


# Now you can use the scores
print(f"Computed cc for {len(clus_scores)} nodes.")